In [1]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'C:\Users\MR068144\Downloads\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import scipy
from matplotlib.gridspec import GridSpec
from __future__ import print_function
from mailmerge import MailMerge
import shutil
import openpyxl
from scipy.optimize import curve_fit
from time import gmtime, strftime
from ipywidgets import FloatProgress
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-10-01 12:24:42


In [2]:
Years = range(2005,2017)
Last5Y = range(12,17)
# Roadway and Crash Data
IRISPath = r'C:\Users\mr068144\Downloads\IRIS\IL_IRIS.gdb'
IRIS_Route = {y:os.path.join(IRISPath,'HWY'+str(y)+'_route') for y in Years}
IRIS_Table = {y:os.path.join(IRISPath,'HWY'+str(y)+'_table_HCurve') for y in Years}
CrashPath = r'C:\Users\mr068144\Downloads\IL Data\IL Crash Data\IL_Crash.gdb'
CrashData = {y:CrashPath + '\\CrashExtract_' + str(y) + '_GIS' for y in Years}

Project_Dir = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\ILSHSP_Tables'
OutputDir   =  Project_Dir + '\\WorkData\\State_Local_Urban_Rural'
DataDir = Project_Dir + '\\SourceData'

In [3]:
def StateLocal(PGN):
    try:
        PGN = str(PGN)
        return({True:'State',False:'Local'}[PGN[0] == 'S'])
    except:
        return('')
def RuralUrban(PGT):
    try:
        PGT = str(PGT)
        return({True:'Urban',False:'Rural'}[PGT.lower().find('urban')!=-1])
    except:
        return('')
def TraffwayConv(TW1,TW2):
    ConvDict = {
        0:'State Rural',
        1:'State Rural',
        2:'State Rural',
        3:'Local Rural',
        4:'State Rural',
        5:'State Urban',
        6:'State Urban',
        8:'Local Urban',
        9:'State Urban'
    }
    tw = ''
    if not math.isnan(TW1):
        tw = int(TW1)
    if not math.isnan(TW2):
        tw = int(TW2)
    if tw in ConvDict.keys():
        return(ConvDict[tw])
def GetCrashTime(y,m,d,s):
    y += 2000
    h = int(s.split(':')[0])
    mm = int(s.split(':')[1].split(' ')[0])
    ampm = s.split(':')[1].split(' ')[1]
    if ampm == 'PM':
            if h<12:
                h+=12
    return(datetime(int(y),int(m),int(d),int(h),int(mm)))
def ConvertLightCondition(l):
    if l in ['Darkness / Lighted Road','Darkness/ Lighted Road']:
        l = 'Darkness, Lighted Road'
    return(l)

In [4]:
print(strftime("%Y-%m-%d %H:%M:%S"))
#Crash_DF = pd.read_excel(Project_Dir + '\\PreviousFiles\\DataSheets_Crash12to16_KA_v2.xlsx',sheetname='12-16_Crash_KA')
#Crash_DF['KA'] = Crash_DF.Total_killed + Crash_DF.A_injuries
#print(Crash_DF.shape)
Crash_DF = pd.read_excel(Project_Dir + '\\SourceData\\CrashExtract_07to16_KA.xlsx')
Crash_DF['KA'] = Crash_DF.Total_killed + Crash_DF.A_injuries
Crash_DF['Date'] = [GetCrashTime(y,m,d,t) for y,m,d,t in zip(Crash_DF.Crash_Year,Crash_DF.Crash_Month,Crash_DF.Crash_Day,Crash_DF.Time_of_crash)]
Crash_DF = Crash_DF.sort_values('Date')
Crash_DF['LightCondition'] = [ConvertLightCondition(l) for l in Crash_DF.Light_condition]

print(Crash_DF.shape)
Occ_DF = pd.read_excel(Project_Dir + '\\SourceData\\PersonData_07to16_KA.xlsx')
print(Occ_DF.shape)
Veh_DF = pd.read_excel(Project_Dir + '\\SourceData\\VehicleExtract_07to16_KA.xlsx')
Veh_DF.index = Veh_DF.Case_number_CH2M
print(Veh_DF.shape)

EADF = pd.read_excel(Project_Dir + '\\SourceData\EA_Definition.xlsx',index_col=0)

Crash_DF.index = Crash_DF.CaseID_CH2M
Occ_DF.index   = Occ_DF.CaseNum_CH2M
Occ_DF['Pedestrian'] = Occ_DF.Pedestrain
Occ_DF['Crash_Year'] = Occ_DF.Year-2000
Occ_DF['A_injuries'] = [{True:1,False:0}[v==3] for v in Occ_DF['Injury Severity']]
Occ_DF['Total_killed'] = [{True:1,False:0}[v==4] for v in Occ_DF['Injury Severity']]
Occ_DF['KA'] = Occ_DF['Total_killed'] + Occ_DF['A_injuries']
Occ_DF['State_Local_Urban_Rural'] = Crash_DF.State_Local_Urban_Rural.loc[Occ_DF.index]
Occ_DF['District'] = Crash_DF.District.loc[Occ_DF.index]
Occ_DF['Date'] = Crash_DF.Date.loc[Occ_DF.index]
EADF = pd.read_excel(Project_Dir + '\\SourceData\EA_Definition.xlsx',index_col=0)
#for i,r in EADF.iterrows():
    #Occ_DF[r.CrashDataField] = Crash_DF[r.CrashDataField].loc[Occ_DF.index]
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-10-01 12:25:03
(108250, 111)
(137747, 56)
(177145, 58)
2018-10-01 12:30:58


In [5]:
print(strftime("%Y-%m-%d %H:%M:%S"))
FC_Dict = {
    'Interstate (PAS)':'01.Interstate',
        'Freeway and Expressway (Urban Only) (PAS)': '01.Interstate',
        'Other Principal Arterial (PAS)': '03.Principal Arterial – Other ',
        'Minor Arterial (Urban)': '04.Minor Arterial',
        'Minor Arterial (Non-Urban)': '04.Minor Arterial',
        'Major Collector (Non-Urban)': '05.Major Collector',
        'Collector (Urban)': '06.Minor Collector',
        'Minor Collector (Non-Urban)': '06.Minor Collector',
        'Local Road or Street (Urban)': '07.Local',
        'Local Road or Street (Non-Urban)': '07.Local',
        '(UNK)': '10.Unknown'}
Crash_DF['FunctionalClass'] = [FC_Dict[s] for s in Crash_DF.Roadway_functional_class_description]
Crash_DF['Time'] = [datetime.time(datetime(2000,1,1,d.hour,0)).strftime('%I:%M %p') for d in Crash_DF.Date]
TimeOrder = [datetime.time(datetime(2000,1,1,d,0)).strftime('%I:%M %p') for d in range(0,24)]
Crash_DF['DayName'] = [d.weekday_name for d in Crash_DF.Date]
DayOrder = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
Crash_DF['Month']  = [d.strftime('%b') for d in Crash_DF.Date]
MonthOrder = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
CityClass = {0:'Unincorporated',
3:'Chicago',
4:'Pop. Under 2500',
5:'Pop. 2,500 - 5,000',
6:'Pop. 5,000 – 10,000',
7:'Pop. 10,000 – 25,000',
8:'Pop. 25,000 – 50,000',
9:'Pop. Over 50,000'}
Crash_DF['City_class'] = [CityClass[c] for c in Crash_DF.City_Class_Code]

2018-10-01 12:30:58


In [38]:
Occ_DF['UnrestrainedOccupants_ISHSP'] = Occ_DF['Unrestrained_Occupants_ISHSP']
for i,r in EADF[EADF.Acronym.isin(['RD','IN','SA','DF','HV','WZ','TN','IM','OD','YD','MC'])].iterrows():
    Occ_DF[r.CrashDataField] = Crash_DF[r.CrashDataField].loc[Occ_DF.index]

In [37]:
Veh_DF['Crash_Year'] = Veh_DF.Year-2000

In [36]:
for f in ['Intersection_related','Type_of_crash','Time','DayName','LightCondition','Month','Roadway_functional_class_description','Weather','Roadway_surface','Cause1','County_name']:
    Occ_DF[f] = Crash_DF[f].loc[Occ_DF.index]

In [104]:
def CalRed(y1,y2):
    try:
        return(y1-y2)
    except:
        return({True:0,False:100}[y1==y2])
def FormatPercentages(r):
    if type(r.Y12_16)==numpy.float64:
        return('{:,.0%}'.format(abs(r.Y12_16)))
    if type(r.Y12_16)==str:
        if r.Reduction==0:
            return('{}'.format(r.Y12_16))
        else:
            return('')
def FormatReduction(r):
    if type(r.Y12_16)==numpy.float64:
        return('({:,.1%} {} compared to 2010-2014)'.format(abs(r.Reduction),{1:'decrease',-1:'increase'}[np.sign(r.Reduction)]))    
    if type(r.Y12_16)==str:
        if r.Reduction==100:
            return('{}'.format(r.Y12_16))
def LineFun(x, A, B):
    return A*x + B
def QueryBulletPoints(Crash_DF,Occ_DF,AC,Veh_DF):
    r = EADF.loc[EADF[EADF.Acronym==AC].index.item()]
    Res = []
    for y in [[10,11,12,13,14],[15,16,12,13,14]]:
        res = []
        df = Crash_DF[(Crash_DF[r.CrashDataField]=='X') & (Crash_DF.Crash_Year.isin(y))]
        if AC=='RD':
            mf1 = df[df.Nbr_of_Vehicles==1]['KA'].sum()/float(df.KA.sum())
            res.append(mf1)

            df2 = df[(df.Weather.isin(['Rain','Snow']))]
            mf2 = df2.KA.sum()/float(df.KA.sum())
            mf2_1 = df2[df2.SpeedingAggressiveDriver=='X'].KA.sum()/df2.KA.sum()
            mf2_2 = df2[df2.ImpairedDriver=='X'].KA.sum()/df2.KA.sum()
            res.extend([mf2,mf2_1,mf2_2])

            df31 = df[df.State_Local_Urban_Rural.isin(['Local Rural','State Rural'])]
            mf3_1_1 = df31.KA.sum()/df.KA.sum()
            mf3_1_2 = df31[df31.State_Local_Urban_Rural=='Local Rural'].KA.sum()/df31.KA.sum()
            mf3_1_3 = df31[(df31.State_Local_Urban_Rural=='Local Rural') & (df31.Type_of_crash=='Fixed Object')].KA.sum()/df31[(df31.State_Local_Urban_Rural=='Local Rural')].KA.sum()

            df32 = df[df.State_Local_Urban_Rural.isin(['Local Urban','State Urban'])]
            mf3_2_1 = df32.KA.sum()/df.KA.sum()
            mf3_2_2 = df32[df32.State_Local_Urban_Rural=='Local Urban'].KA.sum()/df32.KA.sum()
            mf3_2_3 = df32[(df32.State_Local_Urban_Rural=='Local Urban') & (df32.County_name.isin(['Cook','Will','DuPage']))].KA.sum()/df32[(df32.State_Local_Urban_Rural=='Local Urban')].KA.sum()
            res.extend([mf3_1_1,mf3_1_2,mf3_1_3,mf3_2_1,mf3_2_2,mf3_2_3])

            odf = Occ_DF.loc[df.index]
            mf4 = df.loc[list(set(odf[(odf.UnitNo==1) & (odf.Age<25) & (odf.Age>16)].index))].KA.sum()/df.KA.sum()
            #mf4 = df[(df.YoungerDriver16to20=='X') | (df.YoungerDriver21to25=='X')].KA.sum()/df.KA.sum()
            res.append(mf4)

            df5 = df[df.LightCondition.isin(['Darkness','Darkness, Lighted Road','Dusk','Dawn'])]
            mf5 = df5.KA.sum()/df.KA.sum()
            mf5_1 = df5[df5.LightCondition=='Darkness'].KA.sum()/df5.KA.sum()
            mf5_2 = df5[df5.Day_of_week_name.isin(['Sunday','Saturday'])].KA.sum()/df5.KA.sum()
            mf5_2_1 = df5[(df5.Day_of_week_name.isin(['Sunday','Saturday'])) & (df5.ImpairedDriver=='X')].KA.sum()/df5[df5.Day_of_week_name.isin(['Sunday','Saturday'])].KA.sum()
            res.extend([mf5,mf5_1,mf5_2,mf5_2_1])

            df6 = df[df.Trafficway_description.isin(['Not Divided','Divided, No Median Barrier'])]
            mf6 = df6.KA.sum()/df.KA.sum()
            mf6_1 = df6[(df6.Type_of_crash=='Fixed Object')].KA.sum()/df6.KA.sum()
            df611 = df6[(df6.Type_of_crash=='Fixed Object') & (df6.Alignment.isin(['Curve, Level','Curve On Grade','Curve On Hillcrest']))]
            mf6_1_1 = df611.KA.sum()/df6[(df6.Type_of_crash=='Fixed Object')].KA.sum()
            mf6_1_1_1 = df611[df611.ImpairedDriver=='X'].KA.sum()/df611.KA.sum()
            mf6_1_1_2 = df611[df611.SpeedingAggressiveDriver=='X'].KA.sum()/df611.KA.sum()
            res.extend([mf6,mf6_1,mf6_1_1,mf6_1_1_1,mf6_1_1_2])
        
        if AC=='IN':
            df1 = df[df.State_Local_Urban_Rural.isin(['Local Urban','State Urban'])]
            df2 = df[df.State_Local_Urban_Rural.isin(['Local Rural','State Rural'])]
            mf1 = df1.KA.sum()/float(df.KA.sum())
            res.append(mf1)
            df11 = df1[(df1.Traffic_Device=='Traffic Signal')] 
            mf2 = df11.KA.sum()/float(df1.KA.sum())
            res.append(mf2)
            mf3 = df11[df11.Cause1.isin(['Failing to Yield Right of Way','Failing to yield right-of-way'])].KA.sum()/float(df11.KA.sum())
            res.append(mf3)
            mf4 = df11[df11.Cause1.isin(['Disregarding Traffic Signals'])].KA.sum()/float(df11.KA.sum())
            res.append(mf4)

            mf1 = df2.KA.sum()/float(df.KA.sum())
            res.append(mf1)
            df21 = df2[(df2.Traffic_Device=='Stop Sign/Flasher')] 
            mf2 = df21.KA.sum()/float(df2.KA.sum())
            res.append(mf2)
            mf4 = df21[df21.Cause1.isin(['Disregarding Stop Sign'])].KA.sum()/float(df21.KA.sum())
            res.append(mf4)

            df3 = df[df.Nbr_of_Vehicles==1]
            res.append(df3.KA.sum()/float(df.KA.sum()))
            df31 = df3[df3.Pedestrian=='X']
            res.append(df31.KA.sum()/float(df3.KA.sum()))
            odf = Occ_DF.loc[df31.index]
            mf9 = df31.loc[list(set(odf[(odf.PED_Location=='In Crosswalk')].index))].KA.sum()/df31.KA.sum()
            res.append(mf9)


            df4 = df[df.Nbr_of_Vehicles==2]
            res.append(df4.KA.sum()/float(df.KA.sum()))
            res.append(df4[df4.Type_of_crash=='Angle'].KA.sum()/df4.KA.sum())

            df5 = df[df.LightCondition.isin(['Darkness','Darkness, Lighted Road','Dusk','Dawn'])]
            res.append(df5.KA.sum()/df.KA.sum())
            df51 = df5[df5.LightCondition=='Darkness, Lighted Road']
            res.append(df51.KA.sum()/df5.KA.sum())
            res.append(df51[df51.County_name.isin(['Cook','Will','DuPage'])].KA.sum()/df51.KA.sum())
            res.append(df51[df51.ImpairedDriver=='X'].KA.sum()/df51.KA.sum())
            res.append(df51[df51.YoungerDriver16to20=='X'].KA.sum()/df51.KA.sum())
            res.append(df51[df51.SpeedingAggressiveDriver=='X'].KA.sum()/df51.KA.sum())
            
        if AC=='PD':
            df1 = df[df.State_Local_Urban_Rural.isin(['Local Urban','State Urban'])]
            df11 = df[df.State_Local_Urban_Rural.isin(['State Urban'])]
            df12 = df[df.State_Local_Urban_Rural.isin(['Local Urban'])]
            mf1 = df1.KA.sum()/float(df.KA.sum())
            res.append(mf1)
            mf2 = df11.KA.sum()/float(df1.KA.sum())
            res.append(mf2)
            mf3 = df12.KA.sum()/float(df1.KA.sum())
            res.append(mf3)
            cr = df11.groupby('Crash_Year')['KA'].aggregate(sum)
            A,B = curve_fit(LineFun, cr.index, cr)[0]
            #mf4 = (LineFun(y[0],A,B)-LineFun(y[-1],A,B))/LineFun(y[0],A,B)
            mf40 = {1:'decreased',-1:'increased'}[np.sign(cr[y[0]]-cr[y[-1]])]
            mf4 = (cr[y[0]]-cr[y[-1]])/cr[y[0]]
            mf5 = mf4/(len(y)-1)
            mf51 = {1:'reduction',-1:'increase'}[np.sign(cr[y[0]]-cr[y[-1]])]
            res.extend([mf40,mf4,mf5,mf51])

            df111 = df11[df11.Roadway_functional_class_description.isin(['Minor Arterial (Urban)','Other Principal Arterial (PAS)','Minor Arterial (Non-Urban)','Collector (Urban)'])]
            mf6 = df111.KA.sum()/df11.KA.sum()
            res.append(mf6)
            mf7 = df111[df111.Intersection_related=='Y'].KA.sum()/df111.KA.sum()
            res.append(mf7)
            mf8 = df111[(df111.Intersection_related=='Y') & (df111.Traffic_Device=='Traffic Signal')].KA.sum()/df111[df111.Intersection_related=='Y'].KA.sum()
            res.append(mf8)

            odf = Occ_DF.loc[df111.index]
            mf9 = df111.loc[list(set(odf[(odf.PED_Location!='In Crosswalk')].index))].KA.sum()/df111.KA.sum()
            res.append(mf9)
            mf10 = df111.loc[list(set(odf[(odf.PED_Location=='In Roadway')].index))].KA.sum()/df111.KA.sum()
            res.append(mf10)
            mf11 = df111.loc[list(set(odf[(odf.PED_Location=='In Crosswalk')].index))].KA.sum()/df111.KA.sum()
            res.append(mf11)

            cr = df12.groupby('Crash_Year')['KA'].aggregate(sum)
            A,B = curve_fit(LineFun, cr.index, cr)[0]
            mf120 = {1:'increased',-1:'decreased'}[np.sign(cr[y[-1]]-cr[y[0]])]
            #mf12 = (LineFun(y[-1],A,B)-LineFun(y[0],A,B))/LineFun(y[0],A,B)
            mf12 = (cr[y[-1]]-cr[y[0]])/cr[y[0]]
            mf13 = mf12/(len(y)-1)
            mf131 = {-1:'reduction',1:'increase'}[np.sign(cr[y[-1]]-cr[y[0]])]
            res.extend([mf120,mf12,mf13,mf131])

            df121 = df12[df12.Roadway_functional_class_description.isin(['Minor Arterial (Urban)','Other Principal Arterial (PAS)','Minor Arterial (Non-Urban)','Collector (Urban)'])]
            mf14 = df121.KA.sum()/df12.KA.sum()
            res.append(mf14)

            mf15 = df121[df121.Intersection_related=='Y'].KA.sum()/df121.KA.sum()
            res.append(mf15)
            mf16 = df121[(df121.Intersection_related=='Y') & (df121.Traffic_Device=='Traffic Signal')].KA.sum()/df121[df121.Intersection_related=='Y'].KA.sum()
            res.append(mf16)

            odf = Occ_DF.loc[df121.index]
            mf17 = df121.loc[list(set(odf[(odf.PED_Location!='In Crosswalk')].index))].KA.sum()/df121.KA.sum()
            res.append(mf17)
            mf18 = df121.loc[list(set(odf[(odf.PED_Location=='In Roadway')].index))].KA.sum()/df121.KA.sum()
            res.append(mf18)
            mf19 = df121.loc[list(set(odf[(odf.PED_Location=='In Crosswalk')].index))].KA.sum()/df121.KA.sum()
            res.append(mf19)

            mf20 = df121[df121.Cause1.isin(['Failing to Yield Right of Way','Failing to yield right-of-way'])].KA.sum()/df121.KA.sum()
            res.append(mf20)

            odf = Occ_DF.loc[df1.index]
            mf21 = df1.loc[list(set(odf[(odf.PED_Location.isin(['In Crosswalk','In Roadway']))].index))].KA.sum()/df1.KA.sum()
            res.append(mf21)

            df2 = df[df.State_Local_Urban_Rural.isin(['State Rural'])]
            cr = df2.groupby('Crash_Year')['KA'].aggregate(sum)
            A,B = curve_fit(LineFun, cr.index, cr)[0]
            #mf22 = LineFun(y[-1],A,B)/LineFun(y[0],A,B)
            mf22 = cr[y[-1]]/cr[y[0]]
            res.append(mf22)

            mf23 = df[df.County_name.isin(['Cook','Will','DuPage'])].KA.sum()/df.KA.sum()
            res.append(mf23)
            
        if AC=='PC':
            odf = Occ_DF.loc[df.index]
            odf = odf[odf.Age<26]
            res.append(df.loc[list(set(odf.index))].KA.sum()/df.KA.sum())
            res.append(odf[odf.Sex=='M'].KA.sum()/odf.KA.sum())
            res.append(odf[odf.Sex=='F'].KA.sum()/odf.KA.sum())

            df1 = df[df.State_Local_Urban_Rural.isin(['Local Urban','State Urban'])]
            df11 = df[df.State_Local_Urban_Rural.isin(['State Urban'])]
            df12 = df[df.State_Local_Urban_Rural.isin(['Local Urban'])]
            res.append(df1.KA.sum()/df.KA.sum())
            res.append(df11.KA.sum()/df1.KA.sum())
            res.append(df12.KA.sum()/df1.KA.sum())

            cr = df11.groupby('Crash_Year')['KA'].aggregate(sum)
            A,B = curve_fit(LineFun, cr.index, cr)[0]
            mf120 = {1:'increased',-1:'decreased'}[np.sign(cr[y[-1]]-cr[y[0]])]
            #mf12 = (LineFun(y[-1],A,B)-LineFun(y[0],A,B))/LineFun(y[0],A,B)
            mf12 = abs(cr[y[-1]]-cr[y[0]])/cr[y[0]]
            mf13 = mf12/(len(y)-1)
            mf131 = {-1:'reduction',1:'increase'}[np.sign(cr[y[-1]]-cr[y[0]])]
            res.extend([mf120,mf12,mf13,mf131])

            df121 = df11[df11.Roadway_functional_class_description.isin(['Minor Arterial (Urban)','Other Principal Arterial (PAS)','Minor Arterial (Non-Urban)','Collector (Urban)'])]
            mf14 = df121.KA.sum()/df11.KA.sum()
            res.append(mf14)

            mf15 = df121[df121.Intersection_related=='Y'].KA.sum()/df121.KA.sum()
            res.append(mf15)
            mf16 = df121[(df121.Intersection_related=='Y') & (df121.Traffic_Device=='Traffic Signal')].KA.sum()/df121[df121.Intersection_related=='Y'].KA.sum()
            res.append(mf16)

            cr = df12.groupby('Crash_Year')['KA'].aggregate(sum)
            A,B = curve_fit(LineFun, cr.index, cr)[0]
            mf120 = {1:'increased',-1:'decreased'}[np.sign(cr[y[-1]]-cr[y[0]])]
            #mf12 = (LineFun(y[-1],A,B)-LineFun(y[0],A,B))/LineFun(y[0],A,B)
            mf12 = abs(cr[y[-1]]-cr[y[0]])/cr[y[0]]
            mf13 = mf12/(len(y)-1)
            mf131 = {-1:'reduction',1:'increase'}[np.sign(cr[y[-1]]-cr[y[0]])]
            res.extend([mf120,mf12,mf13,mf131])

            df121 = df12[df12.Roadway_functional_class_description.isin(['Minor Arterial (Urban)','Other Principal Arterial (PAS)','Minor Arterial (Non-Urban)','Collector (Urban)'])]
            mf14 = df121.KA.sum()/df12.KA.sum()
            res.append(mf14)

            mf15 = df121[df121.Intersection_related=='Y'].KA.sum()/df121.KA.sum()
            res.append(mf15)
            mf16 = df121[(df121.Intersection_related=='Y') & (df121.Traffic_Device=='Traffic Signal')].KA.sum()/df121[df121.Intersection_related=='Y'].KA.sum()
            res.append(mf16)
            mf3 = df121[df121.Cause1.isin(['Failing to Yield Right of Way','Failing to yield right-of-way'])].KA.sum()/float(df121.KA.sum())
            res.append(mf3)

            res.append(df[df.Day_of_week_name.isin(['Tuesday'])].KA.sum()/df.KA.sum())
            res.append(df[df.Day_of_week_name.isin(['Wednesday'])].KA.sum()/df.KA.sum())
            res.append(df[df.Day_of_week_name.isin(['Friday'])].KA.sum()/df.KA.sum())
            res.append(df[df.Day_of_week_name.isin(['Tuesday','Wednesday','Friday'])].KA.sum()/df.KA.sum())
            res.append(df[(df.Day_of_week_name.isin(['Friday'])) & (df.Time.isin(TimeOrder[12:17]))].KA.sum()/df[df.Day_of_week_name.isin(['Friday'])].KA.sum())
            res.append(df[(df.Day_of_week_name.isin(['Wednesday'])) & (df.Time.isin(TimeOrder[18:]))].KA.sum()/df[df.Day_of_week_name.isin(['Wednesday'])].KA.sum())

            res.append(df[df.Month.isin(['Jun','Jul','Aug'])].KA.sum()/df.KA.sum())
            
        if AC=='SA':
            df1 = df[df.State_Local_Urban_Rural.isin(['Local Urban','State Urban'])]
            res.append(df1.KA.sum()/df.KA.sum())
            df11 = df[df.State_Local_Urban_Rural.isin(['State Urban'])]
            res.append(df11.KA.sum()/df1.KA.sum())
            df111 = df11[df11.Roadway_functional_class_description.isin(['Minor Arterial (Urban)','Other Principal Arterial (PAS)','Minor Arterial (Non-Urban)','Collector (Urban)'])]
            res.append(df111.KA.sum()/df11.KA.sum())
            df112 = df11[df11.Roadway_functional_class_description.isin(['Interstate (PAS)','Freeway and Expressway (Urban Only) (PAS)'])]
            res.append(df112.KA.sum()/df11.KA.sum())
            res.append(df112[(df112.Type_of_crash=='Fixed Object')].KA.sum()/df112.KA.sum())
            
            res.append(df111[df111.Intersection_related=='Y'].KA.sum()/df111.KA.sum())
            res.append(df111[df111.YoungerDriver16to20=='X'].KA.sum()/df111.KA.sum())
            res.append(df111[df111.Type_of_crash=='Rear End'].KA.sum()/df111.KA.sum())
            
            res.append(df[df.RoadwayDeparture=='X'].KA.sum()/df.KA.sum())
            res.append(df[df.YoungerDriver16to20=='X'].KA.sum()/df.KA.sum())
            res.append(df[df.ImpairedDriver=='X'].KA.sum()/df.KA.sum())
            
            odf = Occ_DF.loc[df[df.ImpairedDriver=='X'].index]
            res.append(df.loc[list(set(odf[odf.Sex=='M'].index))].KA.sum()/df[df.ImpairedDriver=='X'].KA.sum())
            odf = Occ_DF.loc[df[df.RoadwayDeparture=='X'].index]
            res.append(df.loc[list(set(odf[odf.Sex=='M'].index))].KA.sum()/df[df.RoadwayDeparture=='X'].KA.sum())
            
            df2 = df[df.Roadway_surface.isin(['Ice','Sand / Mud / Dirt','Sand, Mud, Dirt','Snow or Slush','Wet'])]
            res.append(df2.KA.sum()/df.KA.sum())
            res.append(df2[df2.Type_of_crash=='Rear End'].KA.sum()/df2.KA.sum())
            res.append(df2[df2.Type_of_crash=='Fixed Object'].KA.sum()/df2.KA.sum())
            res.append(df2[df2.Alignment.isin(['Curve, Level','Curve On Grade','Curve On Hillcrest'])].KA.sum()/df2.KA.sum())
            res.append(df[df.Time.isin(TimeOrder[12:18])].KA.sum()/df.KA.sum())            
            
        if AC=='UO':
            UOC = ['Child Restraint Not Used',
             'Child Restraint Used Improperly',
             'Helmet Not Used',
             'None Present',
             'Seat Belts Not Used',
             'Unknown/NA']            
            odf = Occ_DF[(Occ_DF.Unrestrained_Occupants_ISHSP=='X') & (Occ_DF.Crash_Year.isin(y))]
            odf1 = odf[odf.Safety_Equipment.isin(['Seat Belts Not Used'])]
            mf1 = odf1.KA.sum()/odf.KA.sum()
            res.append(mf1)
            res.append(1-mf1)

            res.append(odf[(odf['Person Type']==1) & (odf.Age>=16) & (odf.Age<=20)].KA.sum()/odf.KA.sum())
            res.append(odf[(odf['Person Type']==1) & (odf.Age>=21) & (odf.Age<=25)].KA.sum()/odf.KA.sum())
            res.append(odf[(odf['Person Type']==1) & (odf.Age>=16) & (odf.Age<=25)].KA.sum()/odf.KA.sum())
            
            odf1 = odf[odf['Person Type']==1]
            odf2 = odf[odf['Person Type']==7]
            res.append(odf1.KA.sum()/odf.KA.sum())
            res.append(odf2.KA.sum()/odf.KA.sum())
            res.append(odf1[odf1.Sex=='M'].KA.sum()/odf1.KA.sum())
            res.append(odf1[odf1.Sex=='F'].KA.sum()/odf1.KA.sum())
            res.append(odf2[odf2.Sex=='M'].KA.sum()/odf2.KA.sum())
            res.append(odf2[odf2.Sex=='F'].KA.sum()/odf2.KA.sum())
            
            odf1 = odf[odf.State_Local_Urban_Rural.isin(['Local Urban','State Urban'])]
            odf11 = odf[odf.State_Local_Urban_Rural.isin(['State Urban'])]
            odf12 = odf[odf.State_Local_Urban_Rural.isin(['Local Urban'])]
            res.append(odf1.KA.sum()/odf.KA.sum())
            res.append(odf11.KA.sum()/odf1.KA.sum())
            res.append(odf12.KA.sum()/odf1.KA.sum())

            cr = odf11.groupby('Crash_Year')['KA'].aggregate(sum)
            mf120 = {1:'increased',-1:'decreased'}[np.sign(cr[y[-1]]-cr[y[0]])]
            #mf12 = (LineFun(y[-1],A,B)-LineFun(y[0],A,B))/LineFun(y[0],A,B)
            mf12 = abs(cr[y[-1]]-cr[y[0]])/cr[y[0]]
            mf13 = mf12/(len(y)-1)
            mf131 = {-1:'reduction',1:'increase'}[np.sign(cr[y[-1]]-cr[y[0]])]
            res.extend([mf120,mf12,mf13,mf131])
            
            cr = odf12.groupby('Crash_Year')['KA'].aggregate(sum)
            mf120 = {1:'increased',-1:'decreased'}[np.sign(cr[y[-1]]-cr[y[0]])]
            #mf12 = (LineFun(y[-1],A,B)-LineFun(y[0],A,B))/LineFun(y[0],A,B)
            mf12 = abs(cr[y[-1]]-cr[y[0]])/cr[y[0]]
            mf13 = mf12/(len(y)-1)
            mf131 = {-1:'reduction',1:'increase'}[np.sign(cr[y[-1]]-cr[y[0]])]
            res.extend([mf120,mf12,mf13,mf131])
            
            df12 = odf[odf.State_Local_Urban_Rural.isin(['Local Urban'])]
            df121 = df12[df12.Roadway_functional_class_description.isin(['Minor Arterial (Urban)','Other Principal Arterial (PAS)','Minor Arterial (Non-Urban)','Collector (Urban)'])]
            df122 = df121[df121.Intersection_related!='Y']
            df123 = df122[df122.Type_of_crash.isin(['Fixed Object','Head On','Overturned','Sideswipe Opposite Direction'])]
            res.append(df121.KA.sum()/df12.KA.sum())
            res.append(df122.KA.sum()/df121.KA.sum())
            res.append(df123.KA.sum()/df122.KA.sum())
            res.append(df123[df123.Type_of_crash.isin(['Fixed Object'])].KA.sum()/df123.KA.sum())
            
            df12 = df[df.State_Local_Urban_Rural.isin(['State Urban'])]
            df121 = df12[df12.Roadway_functional_class_description.isin(['Minor Arterial (Urban)','Other Principal Arterial (PAS)','Minor Arterial (Non-Urban)','Collector (Urban)'])]
            df122 = df121[df121.Intersection_related!='Y']
            df123 = df122[df122.Type_of_crash.isin(['Fixed Object','Head On','Overturned','Sideswipe Opposite Direction'])]
            res.append(odf.loc[df121.index].KA.sum()/odf.loc[df12.index].KA.sum())
            res.append(odf.loc[df122.index].KA.sum()/odf.loc[df121.index].KA.sum())
            res.append(odf.loc[df123.index].KA.sum()/odf.loc[df122.index].KA.sum())
            res.append(odf.loc[df123[df123.Type_of_crash.isin(['Fixed Object'])].index].KA.sum()/odf.loc[df123.index].KA.sum())

            df3 = df[df.Day_of_week_name.isin(['Saturday','Sunday','Friday'])]
            res.append(odf.loc[df3.index].KA.sum()/odf.KA.sum())
            df3 = df[(df.Day_of_week_name.isin(['Saturday','Sunday']))]
            res.append(odf.loc[df3[(df3.Time.isin(TimeOrder[0:6]))].index].KA.sum()/odf.loc[df3.index].KA.sum())
            
            res.append(odf[odf.Month.isin(['Apr','May'])].KA.sum()/odf.KA.sum())
            res.append(odf[odf.Vehicle_Type.isin(['Passenger'])].KA.sum()/odf.KA.sum())
            
        if AC=='IM':
            odf = Occ_DF.loc[df.index]
            odf1 = odf[odf.County_name.isin(['Cook','Will','DuPage'])]
            res.append(odf1.KA.sum()/odf.KA.sum())
            res.append(odf1[odf1.RoadwayDeparture=='X'].KA.sum()/odf1.KA.sum())
            res.append(odf1[odf1.SpeedingAggressiveDriver=='X'].KA.sum()/odf1.KA.sum())
            res.append(odf1[odf1.Unrestrained_Occupants_ISHSP=='X'].KA.sum()/odf1.KA.sum())
            
            res.append(df[df.Day_of_week_name.isin(['Saturday','Sunday'])].KA.sum()/df.KA.sum())

            L1 = TimeOrder[0:6]
            L2 = TimeOrder[20:25]
            L3 = L1+L2
            df2 = df[df.Time.isin(L3)]
            res.append(df2.KA.sum()/df.KA.sum())
            res.append(df2[((df2.Day_of_week_name=='Friday') & (df2.Time.isin(L2))) | ((df2.Day_of_week_name=='Saturday') & (df2.Time.isin(L1)))].KA.sum()/df2.KA.sum())
            res.append(df2[((df2.Day_of_week_name=='Saturday') & (df2.Time.isin(L2))) | ((df2.Day_of_week_name=='Sunday') & (df2.Time.isin(L1)))].KA.sum()/df2.KA.sum())
            res.append(df2[((df2.Day_of_week_name=='Sunday') & (df2.Time.isin(L2))) | ((df2.Day_of_week_name=='Monday') & (df2.Time.isin(L1)))].KA.sum()/df2.KA.sum())

            df1 = df[df.State_Local_Urban_Rural.isin(['Local Urban','State Urban'])]
            res.append(df1.KA.sum()/df.KA.sum())
            df11 = df1[df1.Roadway_functional_class_description.isin(['Minor Arterial (Urban)','Other Principal Arterial (PAS)','Minor Arterial (Non-Urban)','Collector (Urban)'])]
            res.append(df11.KA.sum()/df1.KA.sum())
            df111 = df11[df11.State_Local_Urban_Rural.isin(['Local Urban'])]
            res.append(df111.KA.sum()/df1.KA.sum())
            df1111 = df111[df111.RoadwayDeparture=='X']
            res.append(df1111.KA.sum()/df111.KA.sum())
            res.append(df111[df111.Intersection_related=='Y'].KA.sum()/df111.KA.sum())
            res.append(df1111[df1111.Type_of_crash=='Fixed Object'].KA.sum()/df1111.KA.sum())
            res.append(df1[df1.Roadway_functional_class_description.isin(['Interstate (PAS)','Freeway and Expressway (Urban Only) (PAS)'])].KA.sum()/df1.KA.sum())
                            
            df1 = df[df.State_Local_Urban_Rural.isin(['Local Rural','State Rural'])]
            df11 = df1[df1.Roadway_functional_class_description.isin(['Minor Arterial (Urban)',
                                                                      'Major Collector (Non-Urban)',
                                                                      'Other Principal Arterial (PAS)',
                                                                      'Minor Arterial (Non-Urban)',
                                                                      'Minor Collector (Non-Urban)',
                                                                      'Collector (Urban)'])]
            res.append(df11.KA.sum()/df1.KA.sum())
            df12 = df11[df11.State_Local_Urban_Rural.isin(['Local Rural'])]
            res.append(df12.KA.sum()/df11.KA.sum())
            
            df111 = df11[df11.RoadwayDeparture=='X']
            res.append(df111.KA.sum()/df11.KA.sum())
            res.append(df111[df111.Type_of_crash=='Fixed Object'].KA.sum()/df111.KA.sum())
            res.append(df111[df111.UnrestrainedOccupants_ISHSP=='X'].KA.sum()/df111.KA.sum())
            res.append(df11[df11.UnrestrainedOccupants_ISHSP=='X'].KA.sum()/df11.KA.sum())
            res.append(df11[df11.SpeedingAggressiveDriver=='X'].KA.sum()/df11.KA.sum())
            
            df111 = df12[df12.RoadwayDeparture=='X']
            res.append(df111.KA.sum()/df12.KA.sum())
            res.append(df111[df111.Type_of_crash=='Fixed Object'].KA.sum()/df111.KA.sum())
            res.append(df111[df111.UnrestrainedOccupants_ISHSP=='X'].KA.sum()/df111.KA.sum())
            
            
        if AC=='DF':
            df1 = df[df.State_Local_Urban_Rural.isin(['Local Urban','State Urban'])]
            res.append(df1.KA.sum()/df.KA.sum())
            df11 = df[df.State_Local_Urban_Rural.isin(['State Urban'])]
            res.append(df11.KA.sum()/df1.KA.sum())
            df12 = df[df.State_Local_Urban_Rural.isin(['Local Urban'])]
            res.append(df12.KA.sum()/df1.KA.sum())

            cr = df11.groupby('Crash_Year')['KA'].aggregate(sum)
            mf120 = {1:'increased',-1:'decreased'}[np.sign(cr[y[-1]]-cr[y[0]])]
            #mf12 = (LineFun(y[-1],A,B)-LineFun(y[0],A,B))/LineFun(y[0],A,B)
            mf12 = abs(cr[y[-1]]-cr[y[0]])/cr[y[0]]
            mf13 = mf12/(len(y)-1)
            mf131 = {-1:'reduction',1:'increase'}[np.sign(cr[y[-1]]-cr[y[0]])]
            res.extend([mf120,mf12,mf13,mf131])
            
            cr = df12.groupby('Crash_Year')['KA'].aggregate(sum)
            mf120 = {1:'increased',-1:'decreased'}[np.sign(cr[y[-1]]-cr[y[0]])]
            #mf12 = (LineFun(y[-1],A,B)-LineFun(y[0],A,B))/LineFun(y[0],A,B)
            mf12 = abs(cr[y[-1]]-cr[y[0]])/cr[y[0]]
            mf13 = mf12/(len(y)-1)
            mf131 = {-1:'reduction',1:'increase'}[np.sign(cr[y[-1]]-cr[y[0]])]
            res.extend([mf120,mf12,mf13,mf131])
            
            df121 = df12[df12.Roadway_functional_class_description.isin(['Minor Arterial (Urban)','Other Principal Arterial (PAS)','Minor Arterial (Non-Urban)','Collector (Urban)'])]
            res.append(df121.KA.sum()/df12.KA.sum())
            
            df1211 = df121[df121.Intersection_related!='Y']
            res.append(df1211.KA.sum()/df121.KA.sum())
            df1212 = df1211[df1211.RoadwayDeparture=='X']
            res.append(df1212.KA.sum()/df1211.KA.sum())
            res.append(df1212[df1212.Type_of_crash=='Fixed Object'].KA.sum()/df1212.KA.sum())
            
            res.append(df[df.Day_of_week_name.isin(['Friday','Saturday','Thursday'])].KA.sum()/df.KA.sum())

            res.append(df[df.Time.isin([TimeOrder[16]])].KA.sum()/df.KA.sum())
            res.append(df[df.Time.isin(TimeOrder[12:18])].KA.sum()/df.KA.sum())
            res.append(df[df.Month.isin(['Jun','Jul','Aug'])].KA.sum()/df.KA.sum())
            res.append(df[df.Cause1=='Distraction - From Inside Vehicle'].KA.sum()/df.KA.sum())
            res.append(df[df.Cause1=='Physical Condition of Driver'].KA.sum()/df.KA.sum())
            
        if AC=='OD':
            df1 = df[df.State_Local_Urban_Rural.isin(['Local Urban','State Urban'])]
            res.append(df1.KA.sum()/df.KA.sum())
            df11 = df[df.State_Local_Urban_Rural.isin(['State Urban'])]
            df12 = df[df.State_Local_Urban_Rural.isin(['Local Urban'])]

            cr = df11.groupby('Crash_Year')['KA'].aggregate(sum)
            mf120 = {1:'increased',-1:'decreased'}[np.sign(cr[y[-1]]-cr[y[0]])]
            #mf12 = (LineFun(y[-1],A,B)-LineFun(y[0],A,B))/LineFun(y[0],A,B)
            mf12 = abs(cr[y[-1]]-cr[y[0]])/cr[y[0]]
            mf13 = mf12/(len(y)-1)
            mf131 = {-1:'reduction',1:'increase'}[np.sign(cr[y[-1]]-cr[y[0]])]
            res.extend([mf120,mf12])
            
            cr = df12.groupby('Crash_Year')['KA'].aggregate(sum)
            mf120 = {1:'increased',-1:'decreased'}[np.sign(cr[y[-1]]-cr[y[0]])]
            #mf12 = (LineFun(y[-1],A,B)-LineFun(y[0],A,B))/LineFun(y[0],A,B)
            mf12 = abs(cr[y[-1]]-cr[y[0]])/cr[y[0]]
            mf13 = mf12/(len(y)-1)
            mf131 = {-1:'reduction',1:'increase'}[np.sign(cr[y[-1]]-cr[y[0]])]
            res.extend([mf120,mf12])

            res.append(df11.KA.sum()/df1.KA.sum())
            df111 = df11[df11.Roadway_functional_class_description.isin(['Minor Arterial (Urban)',
                                                                      'Major Collector (Non-Urban)',
                                                                      'Other Principal Arterial (PAS)',
                                                                      'Minor Arterial (Non-Urban)',
                                                                      'Minor Collector (Non-Urban)',
                                                                      'Collector (Urban)'])]
            res.append(df111.KA.sum()/df11.KA.sum())
            df1111 = df111[df111.Intersection_related=='Y']
            res.append(df1111.KA.sum()/df111.KA.sum())
            df1112 = df1111[(df1111.Traffic_Device=='Traffic Signal')] 
            res.append(df1112.KA.sum()/df1111.KA.sum())
            df1113 = df1112[(df1112.Collision_type_code.isin([10,15]))] 
            res.append(df1113.KA.sum()/df1112.KA.sum())
            
            res.append(df12.KA.sum()/df1.KA.sum())
            df111 = df12[df12.Roadway_functional_class_description.isin(['Minor Arterial (Urban)',
                                                                      'Major Collector (Non-Urban)',
                                                                      'Other Principal Arterial (PAS)',
                                                                      'Minor Arterial (Non-Urban)',
                                                                      'Minor Collector (Non-Urban)',
                                                                      'Collector (Urban)'])]
            res.append(df111.KA.sum()/df12.KA.sum())
            df1111 = df111[df111.Intersection_related=='Y']
            res.append(df1111.KA.sum()/df111.KA.sum())
            df1112 = df1111[(df1111.Traffic_Device=='Traffic Signal')] 
            res.append(df1112.KA.sum()/df1111.KA.sum())
            df1113 = df1112[(df1112.Collision_type_code.isin([10,15]))] 
            res.append(df1113.KA.sum()/df1112.KA.sum())
            
            df1 = df[df.State_Local_Urban_Rural.isin(['Local Rural','State Rural'])]
            res.append(df1.KA.sum()/df.KA.sum())
            df11 = df[df.State_Local_Urban_Rural.isin(['State Rural'])]
            df12 = df[df.State_Local_Urban_Rural.isin(['Local Rural'])]

            cr = df11.groupby('Crash_Year')['KA'].aggregate(sum)
            mf120 = {1:'increased',-1:'decreased'}[np.sign(cr[y[-1]]-cr[y[0]])]
            #mf12 = (LineFun(y[-1],A,B)-LineFun(y[0],A,B))/LineFun(y[0],A,B)
            mf12 = abs(cr[y[-1]]-cr[y[0]])/cr[y[0]]
            mf13 = mf12/(len(y)-1)
            mf131 = {-1:'reduction',1:'increase'}[np.sign(cr[y[-1]]-cr[y[0]])]
            res.extend([mf120,mf12])
            
            cr = df12.groupby('Crash_Year')['KA'].aggregate(sum)
            mf120 = {1:'increased',-1:'decreased'}[np.sign(cr[y[-1]]-cr[y[0]])]
            #mf12 = (LineFun(y[-1],A,B)-LineFun(y[0],A,B))/LineFun(y[0],A,B)
            mf12 = abs(cr[y[-1]]-cr[y[0]])/cr[y[0]]
            mf13 = mf12/(len(y)-1)
            mf131 = {-1:'reduction',1:'increase'}[np.sign(cr[y[-1]]-cr[y[0]])]
            res.extend([mf120,mf12])
            
            res.append(df[df.Day_of_week_name.isin(['Friday','Wednesday','Thursday'])].KA.sum()/df.KA.sum())
            res.append(df[df.Cause_1_code.isin([5])].KA.sum()/Crash_DF[Crash_DF.Cause_1_code.isin([5])].KA.sum())
            
        if AC=='YD':
            odf = Occ_DF.loc[df.index]
            res.append(odf[(odf.Age<18)].KA.sum()/odf.KA.sum())
            df1 = df[df.Intersection_related=='Y']
            res.append(df1.KA.sum()/df.KA.sum())
            df11 = df1[(df1.Traffic_Device=='Traffic Signal')] 
            res.append(df11.KA.sum()/df1.KA.sum())
            res.append(df[df.RoadwayDeparture=='X'].KA.sum()/df.KA.sum())
            res.append(df[(~df.Day_of_week_name.isin(['Sturday','Sunday'])) & (df.Time.isin(TimeOrder[12:17]))].KA.sum()/df.KA.sum())

            odf1 = odf[(odf['Person Type']==1) & (odf.Age<21)]
            res.append(odf1.KA.sum()/odf.KA.sum())
            res.append(odf1[odf1.Sex=='M'].KA.sum()/odf1.KA.sum())
            res.append(odf1[odf1.Unrestrained_Occupants_ISHSP=='X'].KA.sum()/odf1.KA.sum())
            res.append(Occ_DF[Occ_DF.CaseNum_UnitNo.isin(odf1.CaseNum_UnitNo)].groupby('CaseNum_UnitNo').size().value_counts(True).loc[1])

            res.append(df[df.SpeedingAggressiveDriver=='X'].KA.sum()/df.KA.sum())
            
            
            df1 = df[df.State_Local_Urban_Rural.isin(['Local Urban','State Urban'])]
            res.append(df1.KA.sum()/df.KA.sum())
            df11 = df[df.State_Local_Urban_Rural.isin(['State Urban'])]
            df12 = df[df.State_Local_Urban_Rural.isin(['Local Urban'])]

            df111 = df11[df11.Roadway_functional_class_description.isin(['Minor Arterial (Urban)',
                                                                      'Major Collector (Non-Urban)',
                                                                      'Other Principal Arterial (PAS)',
                                                                      'Minor Arterial (Non-Urban)',
                                                                      'Minor Collector (Non-Urban)',
                                                                      'Collector (Urban)'])]
            res.append(df111.KA.sum()/df11.KA.sum())
            df1111 = df111[df111.Intersection_related=='Y']
            res.append(df1111.KA.sum()/df111.KA.sum())
            df1112 = df1111[(df1111.Traffic_Device=='Traffic Signal')] 
            res.append(df1112.KA.sum()/df1111.KA.sum())
            
            df111 = df12[df12.Roadway_functional_class_description.isin(['Minor Arterial (Urban)',
                                                                      'Major Collector (Non-Urban)',
                                                                      'Other Principal Arterial (PAS)',
                                                                      'Minor Arterial (Non-Urban)',
                                                                      'Minor Collector (Non-Urban)',
                                                                      'Collector (Urban)'])]
            res.append(df111.KA.sum()/df12.KA.sum())
            df1111 = df111[df111.Intersection_related=='Y']
            res.append(df1111.KA.sum()/df111.KA.sum())
            df1112 = df1111[(df1111.Traffic_Device=='Traffic Signal')] 
            res.append(df1112.KA.sum()/df1111.KA.sum())
            
            df1 = df[df.State_Local_Urban_Rural.isin(['Local Rural'])]
            df11 = df1[df1.Roadway_functional_class_description.isin(['Minor Arterial (Urban)',
                                                                      'Major Collector (Non-Urban)',
                                                                      'Other Principal Arterial (PAS)',
                                                                      'Minor Arterial (Non-Urban)',
                                                                      'Minor Collector (Non-Urban)',
                                                                      'Collector (Urban)'])]
            res.append(df11.KA.sum()/df1.KA.sum())
            df12 = df11[df11.RoadwayDeparture=='X']
            res.append(df12.KA.sum()/df11.KA.sum())
            odf1 = Occ_DF.loc[df12.index]
            res.append(odf1[odf1.Unrestrained_Occupants_ISHSP=='X'].KA.sum()/odf1.KA.sum())

            df12 = df1[df1.Roadway_functional_class_description.isin(['Local Road or Street (Non-Urban)',
                                                                      'Local Road or Street (Urban)'])]
            res.append(df12.KA.sum()/df1.KA.sum())
            df13 = df12[df12.RoadwayDeparture=='X']
            res.append(df13.KA.sum()/df12.KA.sum())
            res.append(df13[df13.Type_of_crash=='Fixed Object'].KA.sum()/df13.KA.sum())
            res.append(df13[df13.SpeedingAggressiveDriver=='X'].KA.sum()/df13.KA.sum())
            odf1 = Occ_DF.loc[df13.index]
            res.append(odf1[odf1.Unrestrained_Occupants_ISHSP=='X'].KA.sum()/odf1.KA.sum())
            
        if AC=='HV':
            odf = Occ_DF.loc[df.index]
            idx1 = list(set(Veh_DF[(Veh_DF.VEHT.isin([7,8])) & (Veh_DF.Crash_Year.isin(y))].index))
            res.append(df.loc[idx1].KA.sum()/df.KA.sum())
            idx1 = list(set(Veh_DF[(Veh_DF.VEHT.isin([4,5])) & (Veh_DF.Crash_Year.isin(y))].index))
            res.append(df.loc[idx1].KA.sum()/df.KA.sum())
            #idx1 = list(set(Veh_DF[(Veh_DF.VEHT.isin([6])) & (Veh_DF.Crash_Year.isin(y))].index))
            #res.append(df.loc[idx1].KA.sum()/df.KA.sum())
            df1 = df[df.Roadway_functional_class_description.isin(['Interstate (PAS)','Freeway and Expressway (Urban Only) (PAS)'])]
            res.append(df1.KA.sum()/df.KA.sum())
            idx1 = list(set(Veh_DF[(Veh_DF.VEHT.isin([7,8])) & (Veh_DF.Crash_Year.isin(y))].index))
            res.append(df1.loc[idx1].KA.sum()/df1.KA.sum())
            idx2 = list(set(Veh_DF[(Veh_DF.VEHT.isin([6])) & (Veh_DF.Crash_Year.isin(y))].index))
            res.append(df1.loc[idx2].KA.sum()/df1.KA.sum())
            
            df1 = df.loc[idx1]
            df2 = df1[df1.RoadwayDeparture=='X']
            res.append(df2.KA.sum()/df1.KA.sum())
            res.append(df2[df2.Alignment.isin(['Curve, Level','Curve On Grade','Curve On Hillcrest'])].KA.sum()/df2.KA.sum())
            
            df2 = df[df.Intersection_related=='Y']
            res.append(df2.KA.sum()/df.KA.sum())
            
            df11 = df2[df2.Roadway_functional_class_description.isin(['Minor Arterial (Urban)',
                                                                      'Major Collector (Non-Urban)',
                                                                      'Other Principal Arterial (PAS)',
                                                                      'Minor Arterial (Non-Urban)',
                                                                      'Minor Collector (Non-Urban)',
                                                                      'Collector (Urban)'])]
            res.append(df11.KA.sum()/df2.KA.sum())
            df21 = df2[df2.State_Local_Urban_Rural.isin(['Local Urban','Local Rural'])]
            res.append(df21.KA.sum()/df2.KA.sum())
            idx1 = list(set(Veh_DF[(Veh_DF.VEHT.isin([4,5])) & (Veh_DF.Crash_Year.isin(y))].index))
            res.append(df2.loc[idx1].KA.sum()/df2.KA.sum())
            df3 = df2.loc[idx1]
            res.append(df3[df3.Type_of_crash=='Rear End'].KA.sum()/df3.KA.sum())
            
            res.append(df[df.Alignment.isin(['Curve, Level','Curve On Grade','Curve On Hillcrest'])].KA.sum()/df.KA.sum())
            
        if AC=='MC':
            odf = Occ_DF.loc[df.index]
            res.append(odf[odf.Safety_Equipment=='Helmet Used'].KA.sum()/odf.KA.sum())
            res.append(odf[~odf.Safety_Equipment.isin(['Unknown/NA','Helmet Used'])].KA.sum()/odf.KA.sum())
            res.append(odf[odf.Safety_Equipment=='Unknown/NA'].KA.sum()/odf.KA.sum())
            idx1 = list(set(Veh_DF[(Veh_DF.VEHT.isin([10,11])) & (Veh_DF.UnitNo==1)].index))
            df1 = df.loc[idx1]
            res.append(df1.KA.sum()/df.KA.sum())
            
            odf1 = odf.loc[idx1]
            odf1 = odf1[odf1.UnitNo==1]
            res.append(odf1[~odf1.Safety_Equipment.isin(['Unknown/NA','Helmet Used'])].shape[0]/odf1.shape[0])
            res.append(odf1[odf1.ImpairedDriver=='X'].shape[0]/odf1.shape[0])
            
            df2 = df[df.RoadwayDeparture=='X']
            res.append(df2.KA.sum()/df.KA.sum())
            res.append(df2[df2.SpeedingAggressiveDriver=='X'].KA.sum()/df2.KA.sum())
            res.append(df2[df2.ImpairedDriver=='X'].KA.sum()/df2.KA.sum())
            
            odf1 = odf[odf.Sex=='M']
            res.append(odf1.KA.sum()/odf.KA.sum())
            res.append(odf1.loc[idx1].KA.sum()/odf1.KA.sum())
            
            odf3 = odf[odf.Younger_Driver21to25=='X']
            res.append(odf3.KA.sum()/odf.KA.sum())

            df3 = df[df.State_Local_Urban_Rural.isin(['State Urban','Local Urban'])]
            res.append(df3.KA.sum()/df.KA.sum())
            
            res.append(df[df.Month.isin(['May','Jun','Jul','Aug','Sep'])].KA.sum()/df.KA.sum())            
            res.append(df[df.Month.isin(['Jul'])].KA.sum()/df.KA.sum())            
            
            res.append(df[df.Day_of_week_name.isin(['Sunday','Saturday'])].KA.sum()/df.KA.sum())

            res.append(df[df.Time.isin(TimeOrder[18:24])].KA.sum()/df.KA.sum())

        if AC=='TN':
            df1 = df[(df.Traffic_control_device_code.isin([6,7]))] 
            res.append(df1.KA.sum()/df.KA.sum())
            res.append(df1[df.Traffic_control_device_code.isin([6])].KA.sum()/df.KA.sum())
            res.append(df1[df.Traffic_control_device_code.isin([7])].KA.sum()/df.KA.sum())
            
            df2 = df[df.State_Local_Urban_Rural.isin(['State Urban','Local Urban'])]
            df3 = df[df.State_Local_Urban_Rural.isin(['State Rural','Local Rural'])]
            res.append(df2.KA.sum()/df.KA.sum())
            res.append(df3.KA.sum()/df.KA.sum())

            res.append(df[df.OlderDriver65plus=='X'].KA.sum()/df.KA.sum())
            res.append(df[df.ImpairedDriver=='X'].KA.sum()/df.KA.sum())
            
            df4 = df3[df3.Roadway_functional_class_description.isin(['Minor Arterial (Urban)',
                                                                      'Major Collector (Non-Urban)',
                                                                      'Other Principal Arterial (PAS)',
                                                                      'Minor Arterial (Non-Urban)',
                                                                      'Minor Collector (Non-Urban)',
                                                                      'Collector (Urban)'])]
            res.append(df4.KA.sum()/df3.KA.sum())

            df5 = df3[df3.Roadway_functional_class_description.isin(['Local Road or Street (Non-Urban)',
                                                                      'Local Road or Street (Urban)'])]
            res.append(df5.KA.sum()/df3.KA.sum())
            
            res.append(df4[df4.OlderDriver65plus=='X'].KA.sum()/df4.KA.sum())
            res.append(df4[df4.ImpairedDriver=='X'].KA.sum()/df4.KA.sum())
            odf = Occ_DF.loc[df4.index]
            res.append(odf[odf.Unrestrained_Occupants_ISHSP=='X'].KA.sum()/odf.KA.sum())
            
            res.append(df5[df5.OlderDriver65plus=='X'].KA.sum()/df5.KA.sum())
            res.append(df5[df5.ImpairedDriver=='X'].KA.sum()/df5.KA.sum())
            odf = Occ_DF.loc[df5.index]
            res.append(odf[odf.Unrestrained_Occupants_ISHSP=='X'].KA.sum()/odf.KA.sum())
            
            res.append(df5[df5.Cause_1_code.isin([2])].KA.sum()/df5.KA.sum())
            res.append(df5[df5.Cause_1_code.isin([24])].KA.sum()/df5.KA.sum())
            
            df4 = df2[df2.Roadway_functional_class_description.isin(['Minor Arterial (Urban)',
                                                                      'Major Collector (Non-Urban)',
                                                                      'Other Principal Arterial (PAS)',
                                                                      'Minor Arterial (Non-Urban)',
                                                                      'Minor Collector (Non-Urban)',
                                                                      'Collector (Urban)'])]
            res.append(df4.KA.sum()/df2.KA.sum())

            df5 = df2[df2.Roadway_functional_class_description.isin(['Local Road or Street (Non-Urban)',
                                                                      'Local Road or Street (Urban)'])]
            res.append(df5.KA.sum()/df2.KA.sum())
            
            res.append(df4[df4.OlderDriver65plus=='X'].KA.sum()/df4.KA.sum())
            res.append(df4[df4.ImpairedDriver=='X'].KA.sum()/df4.KA.sum())
            odf = Occ_DF.loc[df4.index]
            res.append(odf[odf.Unrestrained_Occupants_ISHSP=='X'].KA.sum()/odf.KA.sum())
            res.append(df4[df4.Cause_1_code.isin([25])].KA.sum()/df4.KA.sum())
            res.append(df4[df4.Cause_1_code.isin([24])].KA.sum()/df4.KA.sum())
            res.append(df5[df5.ImpairedDriver=='X'].KA.sum()/df5.KA.sum())
            
        if AC=='WZ':    
            df1 = df[df.State_Local_Urban_Rural.isin(['State Urban','State Rural'])]
            df2 = df[df.State_Local_Urban_Rural.isin(['State Urban','Local Urban'])]
            res.append(df1.KA.sum()/df.KA.sum())
            res.append(df2.KA.sum()/df.KA.sum())
            df3 = df[df.State_Local_Urban_Rural.isin(['State Urban'])]
            res.append(df3.KA.sum()/df.KA.sum())
            res.append(df[df.SpeedingAggressiveDriver=='X'].KA.sum()/df.KA.sum())

            df4 = df3[df3.Roadway_functional_class_description.isin(['Interstate (PAS)','Freeway and Expressway (Urban Only) (PAS)'])]
            res.append(df4.KA.sum()/df3.KA.sum())
            df5 = df4[df4.HeavyVehicle=='X']
            res.append(df5.KA.sum()/df4.KA.sum())
            res.append(df5[df5.SpeedingAggressiveDriver=='X'].KA.sum()/df5.KA.sum())
            res.append(df5[df5.Collision_type_code.isin([11])].KA.sum()/df5.KA.sum())

            df6 = df4[df4.UnrestrainedOccupants_ISHSP=='X']
            res.append(df6.KA.sum()/df4.KA.sum())
            res.append(df6[df6.ImpairedDriver=='X'].KA.sum()/df6.KA.sum())

            df7 = df4[df4.OlderDriver65plus=='X']
            res.append(df7.KA.sum()/df4.KA.sum())
            res.append(df7[df.Light_condition_code.isin([2,3,4,5])].KA.sum()/df7.KA.sum())

            df8 = df2[df2.Roadway_functional_class_description.isin(['Minor Arterial (Urban)',
                                                                      'Major Collector (Non-Urban)',
                                                                      'Other Principal Arterial (PAS)',
                                                                      'Minor Arterial (Non-Urban)',
                                                                      'Minor Collector (Non-Urban)',
                                                                      'Collector (Urban)'
                                                                  ])]
            res.append(df8.KA.sum()/df.KA.sum())
            df9 = df8[df8.State_Local_Urban_Rural.isin(['State Urban','State Rural'])]
            res.append(df9.KA.sum()/df8.KA.sum())
            df10 = df9[df9.Intersection_related=='Y']
            res.append(df10.KA.sum()/df9.KA.sum())
            res.append(df10[df10.Collision_type_code.isin([10])].KA.sum()/df10.KA.sum())
            res.append(df10[df10.Collision_type_code.isin([11])].KA.sum()/df10.KA.sum())
            
            df11 = df9[df9.UnrestrainedOccupants_ISHSP=='X']
            res.append(df11.KA.sum()/df9.KA.sum())
            res.append(df11[df11.ImpairedDriver=='X'].KA.sum()/df11.KA.sum())
            
            df11 = df9[df9.RoadwayDeparture=='X']
            res.append(df11.KA.sum()/df9.KA.sum())
            res.append(df11[df11.SpeedingAggressiveDriver=='X'].KA.sum()/df11.KA.sum())
            
        Res.append(res)
    df = pd.DataFrame()
    df['Y10_14'] = Res[0]
    df['Y12_16'] = Res[1]
    df['Reduction'] = [CalRed(y1,y2) for y1,y2 in zip(df.Y10_14,df.Y12_16)]
    df.index = ['mf{}'.format(i+1) for i in range(len(df))]
    return(df)
def UpdateBPDoc(AC,df):
    template = Project_Dir + '\\Template\\{}_MergeDocument.docx'.format(AC)
    document = MailMerge(template)
    document.merge(**{i:FormatPercentages(r) for i,r in df.iterrows()})
    document.merge(**{i+'c':FormatReduction(r) for i,r in df.iterrows() if abs(r.Reduction)>0.02})
    document.write(OutputDir + '\\{}_Bullets.docx'.format(AC))    
    document.close()   

In [81]:
for AC in EADF[EADF.Acronym.isin(['RD','PD','IN','PC','SA','UO','DF','IM','OD','YD','HV','TN','WZ','MC'])].Acronym:
    UpdateBPDoc(AC,QueryBulletPoints(Crash_DF,Occ_DF,AC,Veh_DF))

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:757: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:679: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:680: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [ ]:
# EA Coverage
for year in range(2004,2017):
    cdf = pd.read_csv(r'C:\Users\mr068144\Downloads\IL Data\IL Crash Data\{}_Person_Cleaned_EA.csv'.format(year),encoding='latin-1')
    #cdf = cdf[cdf.Injury_Severity.isin([3,4])]
    def isinAnyEA(r):
        for f in ['RD','PD','IN','PC','SA','UO','DF','IM','OD','YD','HV','TN','WZ','MC']:
            if r[f]!=0:
                return(True)
        return(False)
    cdf['anyEA'] = [isinAnyEA(r) for i,r in cdf.iterrows()]
    #cdf['K'] = [{True:1,False:0}[inj==4] for inj in cdf.Injury_Severity]
    #cdf['A'] = [{True:1,False:0}[inj==3] for inj in cdf.Injury_Severity]
    #cdf['KA'] = [{True:1,False:0}[inj==3 or inj==4] for inj in cdf.Injury_Severity]
    print(year,cdf.groupby('anyEA').size()[True]/cdf.shape[0])